In [4]:
#크롤링 데이터 분석 및 elasticsearch connect 모듈 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import configparser
from datetime import datetime
import json

In [5]:
#elastic cloud connect info load
config = configparser.ConfigParser()
config.read('example.ini')

['example.ini']

In [6]:
#elastic cloud connect
es = Elasticsearch(
    cloud_id=config['ELASTIC']['cloud_id'],
    http_auth=(config['ELASTIC']['user'], config['ELASTIC']['password'])
)

C:\Users\서대표\AppData\Local\Temp\ipykernel_6960\2986887108.py:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [7]:
# category_num -> 장르 이름 : 장르 코드 딕셔너리
# name_mapping -> 장르 이름 : 상명 장르 딕셔너리

category_num = {
                "로맨스":"017001046",
                "BL": "017001064",
                "만화":"017001038",
                "판타지 무협": "017001049",
                "소설": "017001045",
                "경제 경영" : "017001047",                  
                "라이트노벨" : "017001063",
                "에세이 시" : "017001050",
                "인문" : "017001048",
                "사회 정치" : "017001051",
                "자기계발" : "017001054",
                "역사" : "017001052",
                "종교" : "017001053",
                "예술 대중문화" : "017001056",
                "자연과학" : "017001055",
                "가정 살림" : "017001042",
                "건강 취미 여행" : "017001043",
                "어린이 유아" : "017001058",
                "청소년" : "017001059",
                "국어 외국어" : "017001060",
                "IT 모발일" : "017001040",
                "대학교재" : "017001041",
                "수험서 자격증" : "017001061",
                "잡지" : "017001036",
                "해외원서": "017001032",
                "오디오북": "017001032",
                "크레마 (전자책 단말기)": "017001032"
                
}

name_mapping = {
                "로맨스":"문학",
                "BL": "문학",
                "만화":"문학",
                "판타지 무협": "문학",
                "소설": "문학",
                "경제 경영" : "경제/비즈니스",
                "라이트노벨" : "문학",
                "에세이 시" : "에세이/산문",
                "인문" : "인문",
                "사회 정치" : "인문",
                "자기계발" : "인문",
                "역사" : "인문",
                "종교" : "인문",
                "예술 대중문화" : "가정/생활",
                "자연과학" : "자연/과학",
                "가정 살림" : "가정/생활",
                "건강 취미 여행" : "가정/생활",
                "어린이 유아" : "가정/생활",
                "청소년" : "가정/생활",
                "국어 외국어" : "외국어",
                "IT 모발일" : "컴퓨터/인터넷",
                "대학교재" : "강의지원도서",
                "수험서 자격증" : "강의지원도서",
                "잡지" : "국외 eBook",
                "해외원서": "국외 eBook",
                "오디오북": "국외 eBook",
                "크레마 (전자책 단말기)": "국외 eBook"
                
}

In [8]:
#장르 기반 url 생성 및 크롤링 size 조절
def create_url(category_number):
    url = "http://www.yes24.com/24/category/bestseller?CategoryNumber={0}&sumgb=07&FetchSize=80".format(category_number)
    # 80개
    return url

def create_url2(category_number):
    url = "http://www.yes24.com/24/category/bestseller?CategoryNumber={0}&sumgb=07&FetchSize=20&PageNumber=2".format(category_number)
    # 20개
    return url
    

In [9]:
#크롤링 코드
bookss = [] # 책 info 저장 리스트
for key, value in category_num.items():
    rank = 0
    crolling_url = create_url(value) #크롤링 url 생성
    html = requests.get(crolling_url).text # url에 존재하는 text 가져오기
    soup = BeautifulSoup(html, 'html.parser') # html 코드 가져오기
    books = soup.select('td.goodsTxtInfo') # 책 정보를 담고있는 테그
    
    
    

    for book in books:
        title = book.select('p:nth-child(1)')[0].text # 제목
        price = book.select('p:nth-child(3) > span.priceB')[0].text # 가격
        info = book.select('div.aupu')[0].text # 책 저자 | 출판사 | 발행 날짜
        if info.count('|') ==3: # | 구분
            pass
        else:
            writer = info.split('|')[0] # 저자
        
            company = info.split('|')[1] # 출판사

            date = info.split('|')[2] # 발행 날짜
        stars = book.select('p.review > img') # 평점 이미지
        
        star_content = 0 # 내용 평점
        star_edit = 0 # 편집 평점

        if stars==None:
                    
                star_content = 0
                star_edit = 0
                
        else:

        
            for star in stars[:5]:
                
                if star['src'] == 'http://image.yes24.com/sysimage/interface/StarOn.gif':
                    
                    star_content += 1

            for star in stars[5:]:

                if star['src'] == 'http://image.yes24.com/sysimage/interface/StarOn.gif':
        
                    star_edit += 1
        

        
        rank +=1
        bookss.append([rank,title,writer, company, date,price, star_content,star_edit,key]) # 80 add


    # 위와 동일
    crolling_url2 = create_url2(value)
    html2 = requests.get(crolling_url2).text
    soup2 = BeautifulSoup(html2, 'html.parser')
    books2 = soup2.select('td.goodsTxtInfo')

        
    for book in books2:
        title = book.select('p:nth-child(1)')[0].text
        price = book.select('p:nth-child(3) > span.priceB')[0].text
        info = book.select('div.aupu')[0].text
        
        if info.count('|') ==3:
            pass
        else:
            writer = info.split('|')[0]
        
            company = info.split('|')[1]

            date = info.split('|')[2]

       
        stars = book.select('p.review > img')
        
        star_content = 0
        star_edit = 0

        if stars==None:
                    
                star_content = 0
                star_edit = 0
                
        else:

        
            for star in stars[:5]:
                
                if star['src'] == 'http://image.yes24.com/sysimage/interface/StarOn.gif':
                    
                    star_content += 1

            for star in stars[5:]:

                if star['src'] == 'http://image.yes24.com/sysimage/interface/StarOn.gif':
        
                    star_edit += 1
        

   
        rank +=1
        
        bookss.append([rank,title,writer, company, date,price, star_content,star_edit,key]) # 20 add


        



    


In [15]:
#데이터 프레임 생성
df = pd.DataFrame(bookss, columns=['yes24_rank','제목','yes24_writer', 'yes24_company','yes24_date','yes24_price' ,'yes24_star_content','yes24_star_edit','yes24_gener'])


#데이터 전처리-1 : 개행 문자 제거
df['yes24_date'] = df['yes24_date'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl=r' ', regex=True)
df['제목'] = df['제목'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl=r' ', regex=True)
df['yes24_writer'] = df['yes24_writer'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl=r'', regex=True)
df['yes24_company'] = df['yes24_company'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl=r'', regex=True)
df['yes24_date'] = df['yes24_date'].str.strip()

#데이터 전처리-2 : 형식 설정
df['yes24_date'] = df['yes24_date'].str.replace(pat='년 ', repl = '-',regex=True)
df['yes24_date'] = df['yes24_date'].str.replace(pat='월', repl = '',regex=True)
df['yes24_date'] = df['yes24_date'].str.strip()
df['yes24_date']=pd.to_datetime(df['yes24_date'], format='%Y-%m')

df['yes24_writer'] = df['yes24_writer'].str.rstrip("저")

for j in ['원',',']:
    df['yes24_price'] = df['yes24_price'].str.replace(pat=j, repl = '',regex=True)
    df['yes24_price'] = df['yes24_price'].str.strip()
df['yes24_price'] = pd.to_numeric(df['yes24_price'])

#데이터 전처리-3 : 필요없는 요소 제거
for i in ['eBook','EPUB','완결','COMIC','PDF']:
    df['제목'] = df['제목'].str.replace(pat=i, repl=r'', regex=True)


func = lambda x: name_mapping.get(x, x) # name_mapping 딕셔러니 값으로 장르 설정

df['sm_gener'] = df.yes24_gener.map(func) # sm_gener 속성 생성
df['time_stamp'] = datetime.utcnow() #time_stamp 속성 생성

In [16]:
df

,yes24_rank,제목,yes24_writer,yes24_company,yes24_date,yes24_price,yes24_star_content,yes24_star_edit,yes24_gener,sm_gener,time_stamp
0,1,조건 만남 삽화본,문정민,로아,2021-05-01,3000,4,4,로맨스,문학,2023-02-23 12:51:05.287978
1,2,토굴 공주,함초롱,제로노블,2022-12-01,3800,4,4,로맨스,문학,2023-02-23 12:51:05.287978
2,3,세트 오빠 손은 약손 외전 포함 총2권,금나래,텐북,2021-05-01,4200,4,4,로맨스,문학,2023-02-23 12:51:05.287978
3,4,오 담에 핀 꽃 개정판 ...,박영,플로린,2023-02-01,3150,4,4,로맨스,문학,2023-02-23 12:51:05.287978
4,5,세트 재혼 부부 총3권,윤소다,필연매니지먼트,2023-02-01,8370,4,3,로맨스,문학,2023-02-23 12:51:05.287978
...,...,...,...,...,...,...,...,...,...,...,...
1974,8,Empowered Educators ...,KenZeichner,JosseyBass,2017-03-01,43210,0,0,크레마 (전자책 단말기),국외 eBook,2023-02-23 12:51:05.287978
1975,9,The Man Who Lied to His Laptop ...,KenZeichner,JosseyBass,2017-03-01,8500,0,0,크레마 (전자책 단말기),국외 eBook,2023-02-23 12:51:05.287978
1976,10,Call Me by Your Name ...,AndreAciman,FarrarStrausandGiroux,2008-01-01,7500,3,4,크레마 (전자책 단말기),국외 eBook,2023-02-23 12:51:05.287978
1977,11,Holes,LouisSachar,RandomHouseChildrensBooks,2011-06-01,11000,5,5,크레마 (전자책 단말기),국외 eBook,2023-02-23 12:51:05.287978


In [ ]:
df = df[['yes24_rank','제목','yes24_writer', 'yes24_company','yes24_date','yes24_price' ,'yes24_star_content','yes24_star_edit','yes24_gener','sm_gener','time_stamp']]

df.to_csv('yes24.csv', index= False, encoding= 'utf-8')

In [ ]:
json = df.to_json(orient = 'records',force_ascii=False)


In [ ]:
for i in range(len(df)) :
     es.index(index='yes24_best_100_test',#dd
         document = {
         "yes24_rank" : str(df.loc[i].yes24_rank),
         "제목" : str(df.loc[i].제목),
         "yes24_writer": str(df.loc[i].yes24_writer),
         "yes24_company": str(df.loc[i].yes24_company),
         "yes24_date": str(df.loc[i].yes24_date),
         "yes24_price": str(df.loc[i].yes24_price),
         "yes24_star_content": str(df.loc[i].yes24_star_content),
         "yes24_star_edit": str(df.loc[i].yes24_star_edit),
         "yes24_gener": str(df.loc[i].yes24_gener),
         "time_stamp": str(df.loc[i].time_stamp)
 })
 